In [1]:
import numpy as np
import librosa
import os
from glob import glob 
import shutil
import matplotlib.pyplot as plt
import soundfile as sf

1. Set working directory to the original full dataset with the original clips

In [2]:
os.chdir('C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\ESCAPE_II_ONECHANNEL')

2. For our purposes we only want to work with data recorded from devices 306 and 308 (possibly some of the 307 cases) and we only want files ending in .wav
so we can take just those files and put them in an array

In [13]:
files = glob(os.path.join(os.getcwd(), './*/*/*/*.wav'), recursive=True)
workingSet = [f for f in files if os.path.basename(f).startswith('d306') or os.path.basename(f).startswith('d308')]

3. Move target files to a new directory to continue processing

In [16]:
print(len(workingSet))
destination = "C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\DADSONLY_NEW"

count=0
for file in workingSet:
    shutil.copy(file, destination)
    count+=1

print(f"Copied {count} files to {destination}")

264
Copied 264 files to C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\DADSONLY_NEW


4. For each .wav file, separate it into chunks with a duration of 1 second and save each chunk in the same directory with the prefix being the original file name and the postfix being segment_i.wav

In [22]:
%%time
os.chdir("C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\DADSONLY_NEW")

filesToProcess = glob('*.wav')

for w in filesToProcess:
    # Load .wav file using librosa
    y, sr = librosa.load(w, sr=44100)
    
    # Calculate number of 1-second chunks
    duration = librosa.get_duration(y=y, sr=sr)
    num_chunks = int(np.ceil(duration))
    
    # Iterate through each chunk
    for i in range(num_chunks):
        # Set the start and end time for the chunk
        start_time = i
        end_time = min(i + 1, duration)
        
        # Extract audio segment for the chunk
        y_chunk = y[int(start_time * sr):int(end_time * sr)]
        
        # Create output file name
        output_file = f"{os.path.splitext(w)[0]}_chunk{i+1}.wav"
        
        # Write chunk data to a new .wav file using soundfile
        sf.write(output_file, y_chunk, sr)
        
    
    # Delete original .wav file
    os.remove(w)
    print(f"Finished processing {w}, the original file has been deleted")

Chunk 1 saved as d306sA1r01p0120210823_chunk1.wav
Chunk 2 saved as d306sA1r01p0120210823_chunk2.wav
Chunk 3 saved as d306sA1r01p0120210823_chunk3.wav
Chunk 4 saved as d306sA1r01p0120210823_chunk4.wav
Chunk 5 saved as d306sA1r01p0120210823_chunk5.wav
Chunk 6 saved as d306sA1r01p0120210823_chunk6.wav
Chunk 7 saved as d306sA1r01p0120210823_chunk7.wav
Chunk 8 saved as d306sA1r01p0120210823_chunk8.wav
Chunk 9 saved as d306sA1r01p0120210823_chunk9.wav
Chunk 10 saved as d306sA1r01p0120210823_chunk10.wav
Chunk 11 saved as d306sA1r01p0120210823_chunk11.wav
Chunk 12 saved as d306sA1r01p0120210823_chunk12.wav
Chunk 13 saved as d306sA1r01p0120210823_chunk13.wav
Chunk 14 saved as d306sA1r01p0120210823_chunk14.wav
Chunk 15 saved as d306sA1r01p0120210823_chunk15.wav
Chunk 16 saved as d306sA1r01p0120210823_chunk16.wav
Chunk 17 saved as d306sA1r01p0120210823_chunk17.wav
Chunk 18 saved as d306sA1r01p0120210823_chunk18.wav
Chunk 19 saved as d306sA1r01p0120210823_chunk19.wav
Chunk 20 saved as d306sA1r01p0

5. Verify the duration of all the chunks. Remove anything that contains no finite audio, anything shorter than a second still containing finite audio should be padded to reach a 1 second duration. Also, verify that there are no chunks with duration > 1 second.

In [26]:
from librosa.util import fix_length
chunks = glob('*.wav')
sr = 44100 
targetDuration = 1
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration <= 0.05:
        os.remove(c)
        print(f'Removed file {c} which had duration {duration}')
    elif duration < 1:
        padded = fix_length(y, size=targetDuration*sr)
        sf.write(c, padded, sr)
        print(f'Padded {c}, new duration = {librosa.get_duration(y=padded, sr=44100)}')

Removed file d306sA1r01p0420210823_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r01p0520210823_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0120210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0220210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0320210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0420210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0520210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r09p0120210825_chunk61.wav which had duration 2.2675736961451248e-05
Padded d306sA1r09p0220210825_chunk61.wav, new duration = 1.0
Removed file d306sA1r09p0320210825_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r09p0420210825_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r09p0520210825_chunk61.wav wh

Check to make sure each file has duration of exactly 1 second to avoid problems later on

In [29]:
chunks = glob('*.wav')
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration != 1:
        print(f'{c} has duration {duration} and not duration 1')

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


You should now have your complete dataset stored in the directory chosen in step 3. This directory should contain .wav files of duration exactly 1 second which have been created from the original files chosen by you. The original files have since been removed from this directory as to ensure only 1 second long signals are present